# PWL - Lattice

In [1]:
import os
import sys

# import common
import argparse

# import datasets
import numpy as np
import pandas as pd

from tqdm import tqdm

from matplotlib import pyplot as plt

# import estimators as estimators_lib
import itertools
import tensorflow as tf
import tensorflow_lattice as tfl

from query_func import *
from models import *

/var/folders/x9/rlw36ttd3d9dvskjy8fd9t3r0000gn/T/ipykernel_42458/1248846584.py:9: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument("--dataset", type=str, default="test-2", help="Dataset.")
parser.add_argument("--query-size", type=int, default=10000, help="query size")
parser.add_argument("--min-conditions", type=int, default=1, help="min num of conditions")
parser.add_argument("--max-conditions", type=int, default=2, help="max num of conditions")
parser.add_argument("--epochs", type=int, default=10, help="Number of epochs to train for.")
parser.add_argument("--bs", type=int, default=1000, help="Batch size.")
parser.add_argument("--loss", type=str, default="MSE", help="Loss.")
parser.add_argument("--lattice-size", type=int, default=2, help="Lattice size.")
parser.add_argument("--lr", type=float, default=1e-3, help="learning rate")
parser.add_argument("--seed", type=int, default=42, help="Random seed")

try:
    args = parser.parse_args()
except:
    # args = parser.parse_args([])
    args, unknown = parser.parse_known_args()

In [ ]:
def make_directory(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)


OPS = {">": np.greater, "<": np.less, ">=": np.greater_equal, "<=": np.less_equal, "=": np.equal}
FilePath = (
    f"{args.dataset}_{args.query_size}_({args.min_conditions}, {args.max_conditions})_{args.loss}"
)
resultsPath = f"results/{FilePath}"
modelPath = f"saved_models/{FilePath}"
make_directory(resultsPath)
make_directory(modelPath)

In [ ]:
# n_row, n_column 要被table_size 取代

In [ ]:
print("Begin Loading Data ...")
print(f"{args.dataset}.csv")
table = np.loadtxt(f"datasets/{args.dataset}.csv", delimiter=",")
np.savetxt(f"{resultsPath}/original_table.csv", table, delimiter=",")
print("Done.\n")

print("Begin Generating Queries Set...")
table_size = table.shape
n_row, n_column = table_size
rng = np.random.RandomState(args.seed)
query_set = [
    generate_random_query(table, args.min_conditions, args.max_conditions + 1, rng)
    for _ in tqdm(range(args.query_size))
]
print("Done.\n")
print("Begin Intervalization ...")
unique_intervals = column_intervalization(table_size, query_set)
unique_intervals
column_interval_number = count_column_unique_interval(unique_intervals)
print(f"{column_interval_number=}")
print("Done.\n")

In [ ]:
# table = datasets.LoadDataset(f"datasets/{args.dataset}.csv", args.dataset)

# print("Begin Generating Queries Set...")
# rng = np.random.RandomState(args.seed)
# query_set = [
#     GenerateQuery(table, args.min_conditions, args.max_conditions + 1, rng, args.dataset)
#     for _ in tqdm(range(args.query_size))
# ]
# print("Done.\n")


# table_size = table.data.shape
# n_row = table_size[0]
# n_column = table_size[1]
# print("Begin Intervalization ...")
# unique_intervals = dictionary_column_interval(table_size, query_set)
# column_interval_number = count_column_unique_interval(unique_intervals)
# print("Done.\n")
# print(column_interval_number)

In [ ]:
# 修改 x = [sys.maxsize] * n_column     # 这里使用每个col_unique_interval的最后一个元素即可
# 如果使用两个input的话，一个修改为最大，一个修改为最小
train_X = []
for query in query_set:
    x = [sys.maxsize] * n_column  # 这里使用每个col_unique_interval的最后一个元素即可
    idxs, _, vals, _ = query
    for i in range(len(idxs)):
        x[idxs[i]] = vals[i]
    train_X.append(x)
train_X = np.array(train_X).astype(np.float32)
train_Y = np.array([[query[-1]] for query in query_set], dtype=np.float32)

In [ ]:
# train_X = []
# train_Y = []
# for query in query_set:
#     x = [sys.maxsize] * n_column  # 这里使用每个col_unique_interval的最后一个元素即可
#     _, idxs, _, vals, sel = query
#     for i in range(len(idxs)):
#         x[idxs[i]] = vals[i][0]
#     train_X.append(x)
#     train_Y.append(sel)

# train_X = np.array(train_X).astype(np.float32)
# train_Y = np.array(train_Y).astype(np.float32).reshape(-1, 1)

In [ ]:
# make train set unique
# train = np.concatenate((train_X, train_Y), axis=1)
# train = np.unique(train, axis=0)
# train_X, train_Y = np.hsplit(train, [-1])

In [ ]:
# 可以PWL改成三次样条吗

In [ ]:
m = PWLLattice(
    modelPath,
    table_size,
    unique_intervals,
    pwl_keypoints=None,
    lattice_size=args.lattice_size,
)

In [ ]:
m.fit(train_X, train_Y, lr=args.lr, bs=args.bs, epochs=args.epochs, loss=args.loss)

In [ ]:
values = [v for v in unique_intervals.values()]
mesh = np.meshgrid(*values)  # 所有 unique interval 的笛卡尔积网格
grid = np.array(mesh).T.reshape(-1, len(values)).astype(np.float32)

In [ ]:
m.load()
grid_pred = m.predict(grid)

In [ ]:
dataNew = m.generate(grid, grid_pred)
np.savetxt(f"{resultsPath}/generated_table.csv", dataNew, delimiter=",")
Q_error = calculate_Q_error(dataNew, query_set, table_size)
print_Q_error(Q_error, args, resultsPath)

In [ ]:
dataNew

In [ ]:
import matplotlib.pyplot as plt

plt.plot(dataNew[:, 0], dataNew[:, 1], "o")

In [ ]:
break

### 测试简单分布中，使用generate是否能得到正确的分布

In [ ]:
data = np.array([[2, 9], [5, 4], [3, 1], [9, 3], [2, 9], [2, 9], [3, 10], [9, 1], [10, 1], [10, 1]])
values = [range(1, 11), range(1, 11)]
mesh = np.meshgrid(*values)  # 所有unique interval 的笛卡尔积网格
grid = np.array(mesh).T.reshape(-1, len(values)).astype(np.float32)
results = []
df = pd.DataFrame(data, columns=["x", "y"])
for x in range(1, 11):
    for y in range(1, 11):
        count = df[(df["x"] <= x) & (df["y"] <= y)].shape[0]
        results.append(count)
pred = np.array(results).reshape(-1, 1) / df.shape[0]

In [ ]:
n_row = df.shape[0]
n_column = df.shape[1]

In [ ]:
def generate_5(grid, pred=None):
    # 使用 numpy / calculate_query_cardinality_numpy / np.concatenate
    if pred is None:
        pred = m.predict(grid)
    assert pred.shape[0] == grid.shape[0]
    # generate by row, one query may generate several rows
    column_names = [f"col_{i}" for i in range(n_column)]
    # dataNew = pd.DataFrame(columns=column_names)

    count = 0
    ArrayNew = None
    ops = ["<="] * n_column
    pred = (pred * n_row).astype(int)  # Case 1: change 0.8 to 0, 1.8 to 1
    for i in tqdm(range(grid.shape[0])):
        vals = grid[i]
        card = pred[i, 0] - calculate_query_cardinality_numpy(ArrayNew, ops, vals)

        if card >= 1:
            array3 = np.repeat(vals, card).reshape(n_column, card).T
            ArrayNew = array3 if ArrayNew is None else np.concatenate((ArrayNew, array3), axis=0)
            # dataNew = pd.DataFrame(ArrayNew, columns=column_names)
            count += card
            if count > n_row:
                print(
                    f"Reached table max row length({n_row}) in {i}-th row of grid with grid value of {vals}, stop generation."
                )
                break
    else:
        print("Complete table generation")
        # if count < n_row:
        #     print(
        #         f"Reached table max row length({n_row}) in the last row of grid, stop generation."
        #     )
        #     # 如果不足需要补 系统最大值
        #     # dataNew = pd.DataFrame(ArrayNew, columns=column_names)
        return pd.DataFrame(ArrayNew, columns=column_names)
    return pd.DataFrame(ArrayNew, columns=column_names).iloc[:n_row, :]

In [ ]:
dataNew = generate_5(grid, pred)
dataNew

In [ ]:
dataNew.plot(kind="scatter", x="col_0", y="col_1", alpha=0.5)

In [ ]:
# main写成 SingleTable.py
# 关于 class中的load方法，hdf5与h5的区别是什么？应该如何load？load权重还是模型结构一起load？哪一种更好？

### generate_from_batches

In [ ]:
# 确认一下，下面 batch生成的数组格式和grid 的格式是否是相同的，shape

In [ ]:
values = [v for v in unique_intervals.values()]
mesh = np.meshgrid(*values)  # 所有unique interval 的笛卡尔积网格
grid = np.array(mesh).T.reshape(-1, len(values)).astype(np.float32)

In [ ]:
### 修改成 generate_from_batches(grid=None, pred=None)


# 写一个def _generate_grid_batches 方法(下面代码块的重命名）
def generate_batches(values, batch_size):
    iterator = itertools.product(*values)
    while True:
        batch = list(itertools.islice(iterator, batch_size))
        if not batch:
            break
        yield np.array(batch).astype(np.float32)


# unique_intervals = {
#     "A": list(range(1, 101)),
#     "B": list(range(1, 101)),
#     "C": list(range(1, 101)),
#     "D": list(range(1, 101)),
#     "E": list(range(1, 101)),
# }
values = [v for v in unique_intervals.values()]
batch_size = 10000  # 根据实际需要调整批处理大小
total_combinations = np.prod([len(v) for v in values])

processed_batches = []
for batch in tqdm(
    generate_batches(values, batch_size), total=(total_combinations // batch_size) + 1
):
    processed_batch = batch  # 这里可以添加处理逻辑
    processed_batches.append(processed_batch)

# 将所有处理后的批次合并
final_array = np.vstack(processed_batches)

#### 为什么 kl散度是负数，修改 epoch为一个很小的值，看看是否会出现负数

In [ ]:
# 可能与网络的初始化输出值有关？
# 使用 kl散度是否时正确的，如何使用神经网络进行最大似然估计，似然与kl散度之间的关系是什么？

In [ ]:
grid_pred

In [ ]:
train_X

In [ ]:
train_X.shape

In [ ]:
train_Y.shape

In [ ]:
m.predict(train_X)

In [ ]:
# 2024/06/06 update
## 计算 Q-error 的速度：calculate_Q_error > calculate_Q_error_old, 二者的准确性相同, 目前的速度已经很快了
## 生成表的速度：generate_3 > generate > generate_2，之后考虑用numpy的concatenate来替代pd的concat，进一步提升生成速度，第二个思路：如果采用 auto regressive 模型,按列生成是否可以借助 gpu 来加速，
## 生成表的准确性：generate = generate_2 约等于 generate_3(有时候高，有时候低，误差不大，0.1与0.09999999的差别)

# 画图

In [ ]:
# 把plot 整合到 lattice里，或者单独写几个函数

In [ ]:
grid_pred = m.predict(grid)

In [ ]:
plt.figure(figsize=(20, 8))
plt.plot(grid_pred, "bo")

In [ ]:
fig1 = plt.figure(figsize=(15, 8))
ax1 = plt.axes(projection="3d")

# xx = unique_intervals[1]
# yy = unique_intervals[0]
# X, Y = np.meshgrid(xx, yy)

X = grid[:, 1].reshape(column_interval_number[0], column_interval_number[1])  # 这样也可以
Y = grid[:, 0].reshape(column_interval_number[0], column_interval_number[1])
Z = grid_pred.reshape(column_interval_number[0], column_interval_number[1])

ax1.plot_surface(X, Y, Z, cmap="viridis")
plt.show()

In [ ]:
fig2 = plt.figure(figsize=(10, 8))
ax2 = fig2.add_subplot(111)
cs = ax2.contourf(X, Y, Z, cmap="viridis")

# Alternatively, you can manually set the levels
# and the norm:
# lev_exp = np.arange(np.floor(np.log10(z.min())-1),
#                    np.ceil(np.log10(z.max())+1))
# levs = np.power(10, lev_exp)
# cs = ax.contourf(X, Y, z, levs, norm=colors.LogNorm())    # 这个是啥

cbar = fig2.colorbar(cs)  # 让colorbar细粒度更高一点
plt.show()

In [ ]:
# 画一下原生的图做对比，是否需要更光滑
# 变得光滑：
# 1. 数据预处理，缩放，标准化
# 2. 凸函数
# 3. lattice正则器

In [ ]:
# query 对网格的覆盖率 散点图
fig4 = plt.figure(figsize=(10, 10))
xtick = unique_intervals[0]
ytick = unique_intervals[1]
plt.scatter(train_X[:, 0], train_X[:, 1], c="b")
plt.vlines(xtick, min(ytick), max(ytick), colors="green")
plt.hlines(ytick, min(xtick), max(xtick), colors="green")
plt.show()

# Lattice 其它尝试

In [ ]:
# 对比，对query做unique 和 不做unique的误差

In [ ]:
# 对比传入table unique value 和 只传入 unique_intervals的模型优化效果

In [ ]:
# table unique value
data = table.data.to_numpy()
unique_vals = []
for i in range(data.shape[1]):
    unique_vals.append(np.unique(data[:, i]))

In [ ]:
unique_vals

# 尝试 tfl.configs.FeatureConfig

In [ ]:
NUM_EPOCHS = 1000
BATCH_SIZE = 64
LEARNING_RATE = 0.01

In [ ]:
# feat_mins = train_X.min(axis=0)
# feat_maxs = train_X.max(axis=0)
train = np.concatenate((train_X, train_Y), axis=1)
train = np.unique(train, axis=0)
train_X, train_Y = np.hsplit(train, [-1])
df_train = pd.DataFrame(train, columns=[f"col_{i}" for i in range(train.shape[1] - 1)] + ["sel"])

In [ ]:
train_input_fn = tf.compat.v1.estimator.inputs.pandas_input_fn(
    x=df_train,
    y=df_train["sel"],
    batch_size=BATCH_SIZE,
    num_epochs=NUM_EPOCHS,
    shuffle=False,
)

# feature_analysis_input_fn is used for TF Lattice estimators.
feature_analysis_input_fn = tf.compat.v1.estimator.inputs.pandas_input_fn(
    x=df_train,
    y=df_train["sel"],
    batch_size=BATCH_SIZE,
    num_epochs=1,
    shuffle=False,
)

In [ ]:
feature_columns = [
    tf.feature_column.numeric_column("col_0"),
    tf.feature_column.numeric_column("col_1"),
    tf.feature_column.numeric_column("col_2"),
]
model_config = tfl.configs.CalibratedLatticeConfig(
    feature_configs=[
        tfl.configs.FeatureConfig(
            name="col_0",
            lattice_size=2,
            monotonicity="increasing",
            pwl_calibration_num_keypoints=1000,
        ),
        tfl.configs.FeatureConfig(
            name="col_1",
            lattice_size=2,
            monotonicity="increasing",
            pwl_calibration_num_keypoints=1000,
        ),
        tfl.configs.FeatureConfig(
            name="col_1",
            lattice_size=2,
            monotonicity="increasing",
            pwl_calibration_num_keypoints=1000,
        ),
    ]
)
tfl_estimator = tfl.estimators.CannedClassifier(
    feature_columns=feature_columns,
    model_config=model_config,
    feature_analysis_input_fn=feature_analysis_input_fn,
    optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    config=tf.estimator.RunConfig(tf_random_seed=42),
)
tfl_estimator.train(input_fn=train_input_fn)

In [ ]:
from IPython.core.pylabtools import figsize

In [ ]:
def save_and_visualize_lattice(tfl_estimator):
    saved_model_path = tfl_estimator.export_saved_model(
        "/tmp/TensorFlow_Lattice_101/",
        tf.estimator.export.build_parsing_serving_input_receiver_fn(
            feature_spec=tf.feature_column.make_parse_example_spec(feature_columns)
        ),
    )
    model_graph = tfl.estimators.get_model_graph(saved_model_path)
    figsize(8, 8)
    tfl.visualization.draw_model_graph(model_graph)
    return model_graph


_ = save_and_visualize_lattice(tfl_estimator)

In [ ]:
def query_to_lattice_input(table_size, query_set):
    # Traverse all queries to apply the intervalization skill for each column
    n_column = table_size[1]
    x = [sys.maxsize] * n_column
    for i in range(n_column):
        column_interval[i] = set(
            [0, sys.maxsize]
        )  # use set([0, sys.maxsize]) to adapt '>' and '<'.
    for query in query_set:
        col_idxs = query[1]
        vals = query[3]
        for i in range(len(col_idxs)):
            column_interval[col_idxs[i]].add(vals[i][0])
    for k, v in column_interval.items():
        if not v:
            column_interval[k] = [0]
        else:
            column_interval[k] = sorted(list(v))
    return column_interval